In [1]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# tavily検索用APIキーの取得
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 検索結果を返す関数の作成
def get_search_result(question):
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question)
    return json.dumps({"result": response["results"]})

In [3]:
# テスト用コード
ret = get_search_result("東京駅のイベントを教えて")
json.loads(ret)

{'result': [{'title': '東京駅(東京都)周辺のイベント - ウォーカープラス',
   'url': 'https://www.walkerplus.com/event_list/ar0313/sc309880d/',
   'content': '東京駅(東京都)周辺のイベント｜ウォーカープラス 千代田区 中央区 ウォーカープラス 東京駅(東京都)周辺のイベント 東京駅(東京都)周辺のイベント 今日 明日 今週末 千代田区 中央区 アニメ「鬼滅の刃」柱展 ーそして無限城へー終了間近 2024年11月2日(土)～2025年3月2日(日) 江戸にひかり大丸有開催中 2025年1月9日(木)～3月7日(金) 二重橋前駅(東京都), 東京駅(東京都), 大手町駅(東京都), 有楽町駅(東京都), 日比谷駅(東京都) 生誕120年 宮脇綾子の芸術 見た、切った、貼った開催中 2025年1月25日(土)～3月16日(日) 東京駅(東京都), 二重橋前駅(東京都), 大手町駅(東京都), 日本橋駅(東京都), 京橋駅(東京都) 1日限りの光の祭典「アイスキャンドルフェスティバル」と山中湖ダイヤモンド富士と河口湖湖畔 大石公園終了間近 2025年2月23日(日) 二重橋前駅(東京都), 東京駅(東京都), 有楽町駅(東京都), 大手町駅(東京都), 京橋駅(東京都) 丸の内エリアプロジェクションマッピング 東京大浮世絵終了間近 2025年2月21日(金)～25日(火) 二重橋前駅(東京都), 東京駅(東京都), 有楽町駅(東京都), 大手町駅(東京都), 京橋駅(東京都) 二重橋前駅(東京都), 東京駅(東京都), 有楽町駅(東京都), 大手町駅(東京都), 京橋駅(東京都) 二重橋前駅(東京都), 東京駅(東京都), 有楽町駅(東京都), 大手町駅(東京都), 京橋駅(東京都) 大手町駅(東京都), 東京駅(東京都), 二重橋前駅(東京都), 神田駅(東京都), 三越前駅(東京都) Otemachi One Illumination終了間近 2024年11/8(金)～2025年2月下旬 大手町駅(東京都), 竹橋駅(東京都), 神田駅(東京都), 二重橋前駅(東京都), 東京駅(東京都) 京橋エドグラン クリスマスイルミネーション2024終了間近 2024

In [4]:
# ツール定義
def define_tools():
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "指定した質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]

In [5]:
# 言語モデルへの質問を行う関数
def ask_question(question, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": question}],
        tools=tools,
        tool_choice="auto",
    )
    return response

In [6]:
# ツール呼び出しが必要な場合の処理を行う関数
def handle_tool_call(response, question):
    # 関数の実行と結果取得
    tool = response.choices[0].message.tool_calls[0]
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)

    # 関数の実行結果をmessagesに加えて再度言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )
    return response_after_tool_call

In [7]:
# ユーザーからの質問を処理する関数
def process_response(question, tools):
    response = ask_question(question, tools)

    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        final_response = handle_tool_call(response, question)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()

In [8]:
tools = define_tools()

# 言語モデルが直接回答できる質問
question = "東京都と沖縄県はどちらが広いですか？"
response_message = process_response(question, tools)
print(response_message)

東京都（おけいとう）は面積が約2,188.67 km²であるのに対し、沖縄県は約2,276.49 km²です。つまり、沖縄県の方が東京都よりも広いです。


In [9]:
tools = define_tools()

# ツール呼出が必要な質問
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"
response_message = process_response(question, tools)
print(response_message)

最近1ヶ月以内に東京駅で行われているイベントの情報をいくつか紹介します。

1. **東京駅周辺のイベント - ウォーカープラス**
   - 日時: 2025年1月25日（金）～3月30日（日）
   - 内容: 東京駅周辺で開催される様々なイベントが紹介されています。詳細は[こちら](https://www.walkerplus.com/event_list/ar0313/sc309880d/)から。

2. **イベント情報 - 楽しむ東京**
   - 日時: 2025年3月5日（水）～4月20日（日）
   - 内容: Gallery & Bakery Tokyoでの新作個展や、各種アートイベントが予定されています。詳細は[こちら](https://www.enjoytokyo.jp/event/list/area1306/)で確認できます。

3. **東京駅一番街のイベント - 東京駅一番街**
   - 日時: 2025年3月14日（金）～3月27日（木）
   - 内容: MacrossホワイトデーPOP UP SHOPや、特別なイベントが予定されています。詳細は[こちら](https://www.tokyoeki-1bangai.co.jp/event/)でご覧ください。

4. **東京駅周辺の光のイベント**
   - 日時: 2025年1月1日（水）～4月30日（火）
   - 内容: 東京駅近くでの夜間のイルミネーションイベントが開催されています。詳細は[こちら](https://bestcalendar.jp/events/%E6%9D%B1%E4%BA%AC%E9%A7%85)を参照。

これらのイベントは、文化、アート、グルメなど多彩な内容があり、訪れる際の参考にしてください。


In [16]:
# チャットボットへの組み込み
tools = define_tools()

messages=[]

role = "あなたは地理学の教授です。"
messages.append({"role": "system", "content": role})

while(True):
    # ユーザーからの質問を受付
    question = input("メッセージを入力:")
    # 質問が入力されなければ終了
    if question.strip()=="":
        break
    display(f"質問:{question}")

    # メッセージにユーザーからの質問を追加
    messages.append({"role": "user", "content": question.strip()})
    # やりとりが8を超えたら古いメッセージから削除
    if len(messages) > 8:
        del_message = messages.pop(1)

    # 言語モデルに質問
    response_message = process_response(question, tools)

    # メッセージに言語モデルからの回答を追加
    print(response_message, flush=True)
    messages.append({"role": "assistant", "content": response_message})

print("\n---ご利用ありがとうございました！---")

'質問:こんにちは!'

こんにちは！どのようにお手伝いできますか？


'質問:東北6県は？'

東北6県は以下の通りです：

1. 青森県（あおもりけん）
2. 岩手県（いわてけん）
3. 宮城県（みやぎけん）
4. 秋田県（あきたけん）
5. 山形県（やまがたけん）
6. 福島県（ふくしまけん）

これらの県は日本の東北地方に位置しており、それぞれ独自の文化や自然が存在しています。


'質問:宮城県のお土産について検索した結果を教えて'

宮城県のお土産にはいくつかの人気の商品があります。以下にいくつかの例を挙げます。

1. **牛たん** - 名物の牛たんは、宮城県産の厳選された肉を使用しており、多くの店で提供されています。特に、仙台市内には多くの有名牛たん店があります。

2. **笹かまぼこ** - 宮城県の特産品である笹かまぼこは、魚のすり身を使って作られ、特有の風味と食感が楽しめます。詰め合わせのセットも人気です。

3. **ずんだ餅** - 枝豆をペースト状にした「ずんだ」が使われるこの餅は、甘さと豆の風味が絶妙なバランスです。お土産としても人気があります。

4. **かむくらまんじゅう** - 地元の人々に愛されているお菓子で、しっとりとした生地と上品なあんこが特徴です。

5. **フルーツ** - 特に、桃やさくらんぼ、りんごなどの旬の果物は、地元の名産品として評価されています。

6. **地酒** - 宮城県には多くの酒蔵があり、地酒や日本酒もお土産として喜ばれています。

これらの他にも、多くの地元特産品がありますので、訪れた際にはぜひ試してみてください。

---ご利用ありがとうございました！---
